# MNIST DNN으로 된 딥러닝 (Pytorch)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

In [26]:
# 데이터 준비
digits = datasets.load_digits()
X = torch.tensor(digits.images.reshape((len(digits.images), -1)), dtype=torch.float32)
y = torch.tensor(digits.target, dtype=torch.long)

# 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# DataLoader 생성
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [28]:
# DNN 모델 정의
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(64, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = DNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [30]:
# 학습 루프
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # 학습 진행 상황 출력
    if (epoch+1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}] 완료')

Epoch [5/20] 완료
Epoch [10/20] 완료
Epoch [15/20] 완료
Epoch [20/20] 완료


In [32]:
# 평가
model.eval()
correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # 예측값과 실제 라벨 저장
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = correct / total
print(f'Accuracy of the DNN model on the test set: {accuracy:.4f}')

Accuracy of the DNN model on the test set: 0.9667


In [34]:
# 성능 지표 계산
print("\n=== DNN 성능 지표 ===")
print(f"F1 Score: {f1_score(all_labels, all_preds, average='macro'):.4f}")
print(f"Precision: {precision_score(all_labels, all_preds, average='macro'):.4f}")
print(f"Recall: {recall_score(all_labels, all_preds, average='macro'):.4f}")
print("\n=== Confusion Matrix ===")
print(confusion_matrix(all_labels, all_preds))


=== DNN 성능 지표 ===
F1 Score: 0.9663
Precision: 0.9674
Recall: 0.9663

=== Confusion Matrix ===
[[32  0  1  0  0  0  0  0  0  0]
 [ 0 28  0  0  0  0  0  0  0  0]
 [ 0  1 32  0  0  0  0  0  0  0]
 [ 0  0  0 33  0  1  0  0  0  0]
 [ 0  0  0  0 46  0  0  0  0  0]
 [ 0  0  1  0  0 44  1  0  0  1]
 [ 0  0  0  0  0  1 34  0  0  0]
 [ 0  0  0  0  0  0  0 33  0  1]
 [ 0  2  0  0  0  1  0  0 27  0]
 [ 0  0  0  0  0  0  0  1  0 39]]
